In [1]:
import RSA_module

bit_length = int(input("Enter bit_length: "))

public, private = RSA_module.generate_keypair(2**bit_length)

Enter bit_length: 4
181 349


## RSA Encryption

In [2]:
msg = input("\nWrite message: ")
encrypted_msg, encryption_obj = RSA_module.encrypt(msg, public)

print("\nEncrypted message: " + encrypted_msg)


Write message: 7enTropy7

Encrypted message: 56578553244459761073269449223270272212656578


## RSA Decryption

In [3]:
decrypted_msg = RSA_module.decrypt(encryption_obj, private)

print("\nDecrypted message using RSA Algorithm: " + decrypted_msg)


Decrypted message using RSA Algorithm: 7enTropy7


## Shor's Quantum Algorithm

In [4]:
from math import gcd,log
from random import randint
import numpy as np
from qiskit import *

qasm_sim = qiskit.Aer.get_backend('qasm_simulator')

In [5]:
def period(a,N):
    
    available_qubits = 16 
    r=-1   
    
    if N >= 2**available_qubits:
        print(str(N)+' is too big for IBMQX')
    
    qr = QuantumRegister(available_qubits)   
    cr = ClassicalRegister(available_qubits)
    qc = QuantumCircuit(qr,cr)
    x0 = randint(1, N-1) 
    x_binary = np.zeros(available_qubits, dtype=bool)
    
    for i in range(1, available_qubits + 1):
        bit_state = (N%(2**i)!=0)
        if bit_state:
            N -= 2**(i-1)
        x_binary[available_qubits-i] = bit_state    
    
    for i in range(0,available_qubits):
        if x_binary[available_qubits-i-1]:
            qc.x(qr[i])
    x = x0
    
    while np.logical_or(x != x0, r <= 0):
        r+=1
        qc.measure(qr, cr) 
        for i in range(0,3): 
            qc.x(qr[i])
        qc.cx(qr[2],qr[1])
        qc.cx(qr[1],qr[2])
        qc.cx(qr[2],qr[1])
        qc.cx(qr[1],qr[0])
        qc.cx(qr[0],qr[1])
        qc.cx(qr[1],qr[0])
        qc.cx(qr[3],qr[0])
        qc.cx(qr[0],qr[1])
        qc.cx(qr[1],qr[0])
        
        result = execute(qc,backend = qasm_sim, shots=1024).result()
        counts = result.get_counts()
        print(qc)
        results = [[],[]]
        for key,value in counts.items(): #the result should be deterministic but there might be some quantum calculation error so we take the most reccurent output
            results[0].append(key)
            results[1].append(int(value))
        s = results[0][np.argmax(np.array(results[1]))]
    return r

In [6]:
def shors_breaker(N):
    N = int(N)
    while True:
        a=randint(0,N-1)
        g=gcd(a,N)
        if g!=1 or N==1:
            return g,N//g
        else:
            r=period(a,N) 
            if r % 2 != 0:
                continue
            elif pow(a,r//2,N)==-1:
                continue
            else:
                p=gcd(pow(a,r//2)+1,N)
                q=gcd(pow(a,r//2)-1,N)
                if p==N or q==N:
                    continue
                return p,q

In [7]:
def modular_inverse(a,m): 
    a = a % m; 
    for x in range(1, m) : 
        if ((a * x) % m == 1) : 
            return x 
    return 1

In [8]:
N_shor = public[1]
assert N_shor>0,"Input must be positive"
p,q = shors_breaker(N_shor)
phi = (p-1) * (q-1)  
d_shor = modular_inverse(public[0], phi) 

          ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐     »
 q0_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├──■──»
          └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘┌─┴─┐»
 q0_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──┤ X ├»
               └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  └───┘»
 q0_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼───────»
                ║  └╥┘ └┬─┬┘     └───┘         ║                       │       »
 q0_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■───────»
                ║   ║   └╥┘  ┌─┐               ║                               »
 q0_4: |0>──────╫───╫────╫───┤M├───────────────╫───────────────────────────────»
                ║   ║    ║   └╥┘  ┌─┐          ║                               »
 q0_5: |0>──────╫───╫────╫────╫───┤M├──────────╫───────────────────────────────»
          ┌───┐ ║   ║    ║  

          ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐     »
 q1_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├──■──»
          └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘┌─┴─┐»
 q1_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──┤ X ├»
               └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  └───┘»
 q1_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼───────»
                ║  └╥┘ └┬─┬┘     └───┘         ║                       │       »
 q1_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■───────»
                ║   ║   └╥┘  ┌─┐               ║                               »
 q1_4: |0>──────╫───╫────╫───┤M├───────────────╫───────────────────────────────»
                ║   ║    ║   └╥┘  ┌─┐          ║                               »
 q1_5: |0>──────╫───╫────╫────╫───┤M├──────────╫───────────────────────────────»
          ┌───┐ ║   ║    ║  

          ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐     »
 q2_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├──■──»
          └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘┌─┴─┐»
 q2_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──┤ X ├»
               └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  └───┘»
 q2_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼───────»
                ║  └╥┘ └┬─┬┘     └───┘         ║                       │       »
 q2_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■───────»
                ║   ║   └╥┘  ┌─┐               ║                               »
 q2_4: |0>──────╫───╫────╫───┤M├───────────────╫───────────────────────────────»
                ║   ║    ║   └╥┘  ┌─┐          ║                               »
 q2_5: |0>──────╫───╫────╫────╫───┤M├──────────╫───────────────────────────────»
          ┌───┐ ║   ║    ║  

          ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐     »
 q3_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├──■──»
          └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘┌─┴─┐»
 q3_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──┤ X ├»
               └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  └───┘»
 q3_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼───────»
                ║  └╥┘ └┬─┬┘     └───┘         ║                       │       »
 q3_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■───────»
                ║   ║   └╥┘  ┌─┐               ║                               »
 q3_4: |0>──────╫───╫────╫───┤M├───────────────╫───────────────────────────────»
                ║   ║    ║   └╥┘  ┌─┐          ║                               »
 q3_5: |0>──────╫───╫────╫────╫───┤M├──────────╫───────────────────────────────»
          ┌───┐ ║   ║    ║  

          ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐     »
 q4_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├──■──»
          └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘┌─┴─┐»
 q4_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──┤ X ├»
               └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  └───┘»
 q4_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼───────»
                ║  └╥┘ └┬─┬┘     └───┘         ║                       │       »
 q4_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■───────»
                ║   ║   └╥┘  ┌─┐               ║                               »
 q4_4: |0>──────╫───╫────╫───┤M├───────────────╫───────────────────────────────»
                ║   ║    ║   └╥┘  ┌─┐          ║                               »
 q4_5: |0>──────╫───╫────╫────╫───┤M├──────────╫───────────────────────────────»
          ┌───┐ ║   ║    ║  

...

           ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐»
 q24_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├»
           └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘»
 q24_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──»
                └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  »
 q24_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼──»
                 ║  └╥┘ └┬─┬┘     └───┘         ║                       │  »
 q24_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■──»
                 ║   ║   └╥┘  ┌─┐               ║                          »
 q24_4: |0>──────╫───╫────╫───┤M├───────────────╫──────────────────────────»
                 ║   ║    ║   └╥┘  ┌─┐          ║                          »
 q24_5: |0>──────╫───╫────╫────╫───┤M├──────────╫──────────────────────────»
           ┌───┐ ║   ║    ║    ║   └╥┘          ║  ┌─┐                     »

           ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐»
 q25_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├»
           └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘»
 q25_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──»
                └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  »
 q25_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼──»
                 ║  └╥┘ └┬─┬┘     └───┘         ║                       │  »
 q25_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■──»
                 ║   ║   └╥┘  ┌─┐               ║                          »
 q25_4: |0>──────╫───╫────╫───┤M├───────────────╫──────────────────────────»
                 ║   ║    ║   └╥┘  ┌─┐          ║                          »
 q25_5: |0>──────╫───╫────╫────╫───┤M├──────────╫──────────────────────────»
           ┌───┐ ║   ║    ║    ║   └╥┘          ║  ┌─┐                     »

           ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐»
 q26_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├»
           └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘»
 q26_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──»
                └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  »
 q26_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼──»
                 ║  └╥┘ └┬─┬┘     └───┘         ║                       │  »
 q26_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■──»
                 ║   ║   └╥┘  ┌─┐               ║                          »
 q26_4: |0>──────╫───╫────╫───┤M├───────────────╫──────────────────────────»
                 ║   ║    ║   └╥┘  ┌─┐          ║                          »
 q26_5: |0>──────╫───╫────╫────╫───┤M├──────────╫──────────────────────────»
           ┌───┐ ║   ║    ║    ║   └╥┘          ║  ┌─┐                     »

           ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐»
 q27_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├»
           └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘»
 q27_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──»
                └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  »
 q27_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼──»
                 ║  └╥┘ └┬─┬┘     └───┘         ║                       │  »
 q27_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■──»
                 ║   ║   └╥┘  ┌─┐               ║                          »
 q27_4: |0>──────╫───╫────╫───┤M├───────────────╫──────────────────────────»
                 ║   ║    ║   └╥┘  ┌─┐          ║                          »
 q27_5: |0>──────╫───╫────╫────╫───┤M├──────────╫──────────────────────────»
           ┌───┐ ║   ║    ║    ║   └╥┘          ║  ┌─┐                     »

           ┌───┐                               ┌─┐┌───┐┌───┐     ┌───┐┌───┐»
 q28_0: |0>┤ X ├───────────────────────────────┤M├┤ X ├┤ X ├──■──┤ X ├┤ X ├»
           └───┘┌─┐┌───┐     ┌───┐     ┌───┐   └╥┘└───┘└─┬─┘┌─┴─┐└─┬─┘└─┬─┘»
 q28_1: |0>─────┤M├┤ X ├─────┤ X ├──■──┤ X ├────╫────────■──┤ X ├──■────┼──»
                └╥┘└┬─┬┘┌───┐└─┬─┘┌─┴─┐└─┬─┘    ║           └───┘       │  »
 q28_2: |0>──────╫──┤M├─┤ X ├──■──┤ X ├──■──────╫───────────────────────┼──»
                 ║  └╥┘ └┬─┬┘     └───┘         ║                       │  »
 q28_3: |0>──────╫───╫───┤M├────────────────────╫───────────────────────■──»
                 ║   ║   └╥┘  ┌─┐               ║                          »
 q28_4: |0>──────╫───╫────╫───┤M├───────────────╫──────────────────────────»
                 ║   ║    ║   └╥┘  ┌─┐          ║                          »
 q28_5: |0>──────╫───╫────╫────╫───┤M├──────────╫──────────────────────────»
           ┌───┐ ║   ║    ║    ║   └╥┘          ║  ┌─┐                     »

## Cracking RSA using Shor's Algorithm

In [9]:
decrypted_msg = RSA_module.decrypt(encryption_obj, (d_shor,N_shor))

print('\nMessage Cracked using Shors Algorithm: ' + decrypted_msg + "\n")


Message Cracked using Shors Algorithm: 7enTropy7

